In [1]:
import pandas as pd
from datetime import datetime

vivid = pd.read_csv("/Users/Student/Desktop/STA 141B/vividseats_lakers_events.csv")
ticketmaster = pd.read_csv("/Users/Student/Desktop/STA 141B/lakers_remaining_games_with_min_price.csv")
seatgeek = pd.read_csv("/Users/Student/Desktop/STA 141B/seatgeek_events.csv")

#list of all the teams in the nba
teams = ["lakers","celtics","warriors","clippers","spurs","kings","suns","nets",
    "knicks","bulls","jazz","pistons","grizzlies","raptors","hawks","hornets",
    "pelicans","mavericks","bucks","cavaliers","wizards","rockets","nuggets",
    "thunder","timberwolves","magic","heat","76ers","trail blazers","pacers"]

#This looks at the event title name and returns a list of teams in that title
def teamsearch(event):
    event = str(event).lower()
    teamlist = []
    for key in teams:
        if key in event:
            teamlist.append(key)
    return sorted(set(teamlist))

#makes the list found in teamsearch into a string. 
#this also only keeps events with only two teams  
def matchups(event):
    teamsf = teamsearch(event)
    return "_".join(sorted(teamsf)) if len(teamsf) >= 2 else None

#makes a new column in each dataset with the clean titles
vivid["matchup"] = vivid["event_title"].apply(matchups)
ticketmaster["matchup"] = ticketmaster["event_name"].apply(matchups)
seatgeek["matchup"] = seatgeek["title"].apply(matchups)

#adds a year to the date for the datasets that do not have a year
def years(dates):
    month = datetime.strptime(dates, "%b %d").month
    return 2025 if month >= 10 else 2026

#adds a column with the year for vivid
vivid["year"] = vivid["event_date"].apply(years)
#adds a column with the year and date combined 
vivid["new_event_date"] = vivid["event_date"] + " " + vivid["year"].astype(str)

#cleans the date and returns the same format for three datasets
vivid["Date"] = pd.to_datetime(vivid["new_event_date"] + " " + vivid["event_time"],format="%b %d %Y %I:%M%p",errors="coerce").dt.date
ticketmaster["Date"] = pd.to_datetime(ticketmaster["start_date_local"], errors="coerce").dt.date
seatgeek["Date"] = pd.to_datetime(seatgeek["datetime_local"], errors="coerce").dt.date

#makes new columns for the price in each dataset
vivid["vivid_price"] = vivid["price_int"]
ticketmaster["ticketmaster_price"] = ticketmaster["scraped_min_price"]
seatgeek["seatgeek_price"] = seatgeek["lowest price lower bound"]

#makes new dataframes for the columns needed to merge the data
vividdf = vivid[["Date", "matchup", "vivid_price"]]
seatgeekdf = seatgeek[["Date", "matchup", "seatgeek_price"]]

#combines ticketmaster with vividseats and seatgeek
combine = ticketmaster.merge(vividdf,on=["Date", "matchup"],how="left")
combine = combine.merge(seatgeekdf,on=["Date", "matchup"],how="left")

#drops rows and columns that are not needed
combine = combine.drop(columns=['api_min_price','currency','api_max_price','start_date_local','scraped_min_price'])
combine = combine.drop(25)

combine.to_csv("combine_games.csv", index=False)


C:\Users\Student\AppData\Local\Temp\ipykernel_12860\396182063.py:36: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  month = datetime.strptime(dates, "%b %d").month
